# Building a Financial Analyst

## Setup

### Install Packages

In [1]:
# !uv pip install yfinance crewai crewai-tools

In [2]:
#Need to install Docker also
#Check with docker --version on terminal/command line to check if Docker is installed
# export PATH="$PATH:/Applications/Docker.app/Contents/Resources/bin/"

Resolve this issue in CrewAI before proceeding: https://github.com/crewAIInc/crewAI/issues/2606

### API Keys

In [3]:
import os, json, re, getpass
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [4]:
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

In [5]:
#Check for Groq API Key
_set_env("GROQ_API_KEY")

### Libraries

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
import re
import json
import os
import yfinance as yf
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import CodeInterpreterTool, FileReadTool
from pydantic import BaseModel, Field
from IPython.display import display, Markdown

/Users/anantagarwal/projects/generative-ai-2025/.venv/lib/python3.12/site-packages/pydantic/fields.py:1089: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


## Selecting Components

### Chat Model

In [22]:
llm = LLM(
    # model="groq/llama-3.1-8b-instant",
    model="openai/gpt-4o-mini",
    # temperature=0.5,
    # max_completion_tokens=1024,
    # top_p=0.9,
    # stop=None,
    # stream=False,
)

### Structured Output for Query Analysis

**NOTE:**: When defining state in LangGraph, you have two primary options: **TypedDict** and **Pydantic BaseModel**. The best choice depends on your specific needs:

**TypedDict** is a good fit if you're looking for a *lightweight solution without strict runtime type enforcement*. It's simple to use and ideal for smaller applications where the overhead of a more robust system isn't necessary.

For *greater robustness and early error detection*, especially in larger or more complex applications, **Pydantic BaseModel** is the superior choice. Pydantic provides powerful data validation and parsing, catching type-related errors early in the development process and improving the overall reliability of your application.

In [23]:
class QueryAnalysisOutput(BaseModel):
    """Structured output for the query analysis task."""
    symbol: str = Field(..., description="Stock ticker symbol (e.g., TSLA, AAPL).")
    timeframe: str = Field(..., description="Time period (e.g., '1d', '1mo', '1y').")
    action: str = Field(..., description="Action to be performed (e.g., 'fetch', 'plot').")

## Setting up Crew of Agents

In CrewAI, you define **agents** as independent workers, each with a specific job and objective. You then assign them **tasks**, which are detailed instructions on what they need to do to reach that objective. You can also assign **tools** directly as tasks for agents to use.

**Let's first create our CrewAI agents with role, main goal/objective, and backstory/personality.**

### Query Parser Agent

In [24]:
query_parser_agent = Agent(
    role="Stock Data Analyst",
    goal="Extract stock details and fetch required data from this user query: {query}.",
    backstory="You are a financial analyst specializing in stock market data retrieval.",
    llm=llm,
    verbose=True,
    memory=True,
)

### Code Writer Agent

In [25]:
code_writer_agent = Agent(
    role="Senior Python Developer",
    goal="Write Python code to visualize stock data.",
    backstory="""You are a Senior Python developer specializing in stock market data visualization. 
                 You are also a Pandas, Matplotlib and yfinance library expert.
                 You are skilled at writing production-ready Python code""",
    llm=llm,
    verbose=True,
)

### Code Interpreter Agent

**Note:** By default, the Code Interpreter Tool is prompted by CrewAI to run on Docker to safeguard against any usnafe action that the code can take, since it'll run in a container (i.e., `code_interpreter_tool = CodeInterpreterTool()`  or `allow_code_execution=True`).

To bypass this, we can set `code_interpreter_tool = CodeInterpreterTool(unsafe_mode=True)` and use this only when defining the agent (not `allow_code_execution=True`) 

In [26]:
code_interpreter_tool = CodeInterpreterTool(unsafe_mode=True)

code_execution_agent = Agent(
    role="Senior Code Execution Expert",
    goal="Review and execute the generated Python code by code writer agent to visualize stock data.",
    backstory="You are a code execution expert. You are skilled at executing Python code.",
    tools=[code_interpreter_tool],
    # allow_code_execution=True,   # This automatically adds the CodeInterpreterTool
    llm=llm,
    verbose=True,
)

Generating description...
Method called by: CodeInterpreterTool
{'code': {'description': 'Python3 code used to be interpreted in the Docker container. ALWAYS PRINT the final result and the output of the code', 'type': 'str', 'required': True}, 'libraries_used': {'description': 'List of libraries used in the code with proper installing names separated by commas. Example: numpy,pandas,beautifulsoup4', 'type': 'list[str]', 'required': True}}
Tool Name: Code Interpreter
Tool Arguments: {'code': {'description': 'Python3 code used to be interpreted in the Docker container. ALWAYS PRINT the final result and the output of the code', 'type': 'str', 'required': True}, 'libraries_used': {'description': 'List of libraries used in the code with proper installing names separated by commas. Example: numpy,pandas,beautifulsoup4', 'type': 'list[str]', 'required': True}}
Tool Description: Interprets Python3 code strings with a final print statement.
Usage Instructions: To use this tool (Code Interpreter

**Now, let's define the agents' tasks.**

### Query Parser Agent - Task

In [27]:
query_parsing_task = Task(
    description="Analyze the user query and extract stock details.",
    expected_output="A dictionary with keys: 'symbol', 'timeframe', 'action'.",
    output_pydantic=QueryAnalysisOutput,
    agent=query_parser_agent,
)

### Code Writer Agent - Task

In [28]:
code_writer_task = Task(
    description="""Write Python code to visualize stock data based on the inputs from the stock analyst
                   where you would find stock symbol, timeframe and action.""",
    expected_output="A clean and executable Python script file (.py) for stock visualization.",
    agent=code_writer_agent,
)

### Code Interpreter Agent - Task

In [29]:
code_execution_task = Task(
    description="""Review and execute the generated Python code by code writer agent to visualize stock data.""",
    expected_output="A clean and executable Python script file (.py) for stock visualization.",
    agent=code_execution_agent,
)

**Now, let's create crew to manage agents and task workflow.**

In [30]:
crew = Crew(
    agents=[query_parser_agent, code_writer_agent, code_execution_agent],
    tasks=[query_parsing_task, code_writer_task, code_execution_task],
    process=Process.sequential
)

In [31]:
# Run the crew with an example query
result = crew.kickoff(inputs={"query": "Plot YTD stock gain of Tesla"})

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Stock Data Analyst                                                                                      │
│                                                                                                                 │
│  Task: Analyze the user query and extract stock details.                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Stock Data Analyst                                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "symbol": "TSLA",                                                                                            │
│    "timeframe": "YTD",                                                                                          │
│    "action": "Plot stock gain"                                                                                  │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Python Developer                                                                                 │
│                                                                                                                 │
│  Task: Write Python code to visualize stock data based on the inputs from the stock analyst                     │
│                     where you would find stock symbol, timeframe and action.                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Python Developer                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```python                                                                                                      │
│  import yfinance as yf                                                                                          │
│  import pandas as pd                                                                                            │
│  import matplotlib.pyplot as plt                                                                                │
│                                                                                                                 │
│  def plot_stock_gain(symbol: str, timeframe: str):                                                              │
│      # Download stock data                                                                                      │
│      stock_data = yf.download(symbol)                                                                           │
│                                                                                                                 │
│      # Filter Data based on Timeframe                                                                           │
│      if timeframe == "YTD":                                                                                     │
│          start_date = f"{pd.Timestamp.now().year}-01-01"                                                        │
│      else:                                                                                                      │
│          raise ValueError("Unsupported timeframe. Currently only 'YTD' is supported.")                          │
│                                                                                                                 │
│      stock_data = stock_data[stock_data.index >= start_date]                                                    │
│                                                                                                                 │
│      # Calculate the daily percentage gain                                                                      │
│      stock_data['Daily Gain (%)'] = stock_data['Adj Close'].pct_change() * 100                                  │
│                                                                                                                 │
│      # Plotting                                                                                                 │
│      plt.figure(figsize=(14, 7))                                                                                │
│      plt.plot(stock_data.index, stock_data['Daily Gain (%)'], label=f'{symbol} Daily Gain (%)', color='blue')   │
│      plt.title(f'Daily Percentage Gain for {symbol} - Year to Date')                                            │
│      plt.xlabel('Date')                                                                                         │
│      plt.ylabel('Daily Gain (%)')                                                                               │
│      plt.axhline(0, color='red', linestyle='--', lw=1)  # Adding a horizontal line at 0                         │
│      plt.legend()                                                                                               │
│      plt.grid()                                                                                                 │
│      plt.show()                                        

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Code Execution Expert                                                                            │
│                                                                                                                 │
│  Task: Review and execute the generated Python code by code writer agent to visualize stock data.               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/Users/anantagarwal/opt/anaconda3/lib/python3.8/site-packages/six-1.15.0.dist-info/METADATA'



╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Code Execution Expert                                                                            │
│                                                                                                                 │
│  Thought: Thought: I have all the necessary information to execute the Python code for visualizing stock data.  │
│  Now, I will proceed to execute the given code to visualize Tesla's stock gains for the year-to-date            │
│  timeframe.                                                                                                     │
│                                                                                                                 │
│  Using Tool: Code Interpreter                                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"code\": \"import yfinance as yf\\nimport pandas as pd\\nimport matplotlib.pyplot as plt\\n\\ndef plot_sto  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  No result variable found.                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Code Execution Expert                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  import yfinance as yf                                                                                          │
│  import pandas as pd                                                                                            │
│  import matplotlib.pyplot as plt                                                                                │
│                                                                                                                 │
│  def plot_stock_gain(symbol: str, timeframe: str):                                                              │
│      # Download stock data                                                                                      │
│      stock_data = yf.download(symbol)                                                                           │
│                                                                                                                 │
│      # Filter Data based on Timeframe                                                                           │
│      if timeframe == "YTD":                                                                                     │
│          start_date = f"{pd.Timestamp.now().year}-01-01"                                                        │
│      else:                                                                                                      │
│          raise ValueError("Unsupported timeframe. Currently only 'YTD' is supported.")                          │
│                                                                                                                 │
│      stock_data = stock_data[stock_data.index >= start_date]                                                    │
│                                                                                                                 │
│      # Calculate the daily percentage gain                                                                      │
│      stock_data['Daily Gain (%)'] = stock_data['Adj Close'].pct_change() * 100                                  │
│                                                                                                                 │
│      # Plotting                                                                                                 │
│      plt.figure(figsize=(14, 7))                                                                                │
│      plt.plot(stock_data.index, stock_data['Daily Gain (%)'], label=f'{symbol} Daily Gain (%)', color='blue')   │
│      plt.title(f'Daily Percentage Gain for {symbol} - Year to Date')                                            │
│      plt.xlabel('Date')                                                                                         │
│      plt.ylabel('Daily Gain (%)')                                                                               │
│      plt.axhline(0, color='red', linestyle='--', lw=1)  # Adding a horizontal line at 0                         │
│      plt.legend()                                                                                               │
│      plt.grid()                                                                                                 │
│      plt.show()                                                                                                 │
│                                                        

In [32]:
#Print result
print(result.raw)

import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

def plot_stock_gain(symbol: str, timeframe: str):
    # Download stock data
    stock_data = yf.download(symbol)

    # Filter Data based on Timeframe
    if timeframe == "YTD":
        start_date = f"{pd.Timestamp.now().year}-01-01"
    else:
        raise ValueError("Unsupported timeframe. Currently only 'YTD' is supported.")

    stock_data = stock_data[stock_data.index >= start_date]

    # Calculate the daily percentage gain
    stock_data['Daily Gain (%)'] = stock_data['Adj Close'].pct_change() * 100

    # Plotting
    plt.figure(figsize=(14, 7))
    plt.plot(stock_data.index, stock_data['Daily Gain (%)'], label=f'{symbol} Daily Gain (%)', color='blue')
    plt.title(f'Daily Percentage Gain for {symbol} - Year to Date')
    plt.xlabel('Date')
    plt.ylabel('Daily Gain (%)')
    plt.axhline(0, color='red', linestyle='--', lw=1)  # Adding a horizontal line at 0
    plt.legend()
    plt.grid()
    plt.show(